##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 템플릿을 사용하여 TFX 파이프라인 생성하기

Note: We recommend running this tutorial on Google Cloud Vertex AI Workbench. [Launch this notebook on Vertex AI Workbench](https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?q=download_url%3Dhttps%253A%252F%252Fraw.githubusercontent.com%252Ftensorflow%252Ftfx%252Fmaster%252Fdocs%252Ftutorials%252Ftfx%252Ftemplate.ipynb).

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td>     <a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/template"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a>
</td>
<td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tfx/blob/master/docs/tutorials/tfx/template.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a>
</td>
<td>     <a target="_blank" href="https://github.com/tensorflow/tfx/tree/master/docs/tutorials/tfx/template.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a>
</td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/tfx/docs/tutorials/tfx/template.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table></div>

## 시작하기

이 문서에서는 TFX Python 패키지와 함께 제공되는 *템플릿*을 사용하여 TensorFlow Extended(TFX) 파이프라인 생성 지침을 제공합니다. 지침의 상당 부분은 AI Platform Notebooks 인스턴스에서 실행되는 Linux 쉘 명령어입니다. `!`를 사용하여 해당 명령을 호출하는 주피터 노트북 코드 셀이 제공됩니다.

시카고 시에서 출시된 [Taxi Trips 데이터세트](https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew)를 사용하여 파이프라인을 구축할 수 있습니다. 이 파이프라인을 활용하여 데이터세트를 사용하는 자체 파이프라인을 구축할 것을 권장합니다.


## 1단계. 환경 설정

AI Platform Pipeline은 개발 환경을 통해 파이프라인을 구축하고Kubeflow Pipeline 클러스터로 새로 빌드된 파이프라인을 실행합니다.

**참고:** 특정 TensorFlow 버전을 선택하거나 GPU 인스턴스를 선택하려면 AI Platform Notebooks에서 TensorFlow 사전 설치된 인스턴스를 생성하세요.


Install `tfx` python package with `kfp` extra requirement.

In [ ]:
import sys
# Use the latest version of pip.
!pip install --upgrade pip
# Install tfx and kfp Python packages.
!pip install --upgrade "tfx[kfp]<2"

TFX 버전을 확인합니다.

In [ ]:
!python3 -c "from tfx import version ; print('TFX version: {}'.format(version.__version__))"

AI Platform Pipelines에서 TFX는 [Kubeflow Pipelines](https://www.kubeflow.org/docs/pipelines/overview/pipelines-overview/)을 사용하여 호스팅된 Kubernetes 환경에서 실행됩니다.

몇 가지 환경 변수를 설정하여 Kubeflow Pipelines를 사용해 보겠습니다.

우선 GCP 프로젝트 ID를 가져옵니다.

In [ ]:
# Read GCP project id from env.
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
GOOGLE_CLOUD_PROJECT=shell_output[0]
%env GOOGLE_CLOUD_PROJECT={GOOGLE_CLOUD_PROJECT}
print("GCP project ID:" + GOOGLE_CLOUD_PROJECT)

We also need to access your KFP cluster. You can access it in your Google Cloud Console under "AI Platform &gt; Pipeline" menu. The "endpoint" of the KFP cluster can be found from the URL of the Pipelines dashboard, or you can get it from the URL of the Getting Started page where you launched this notebook. Let's create an `ENDPOINT` environment variable and set it to the KFP cluster endpoint. **ENDPOINT should contain only the hostname part of the URL.** For example, if the URL of the KFP dashboard is `https://1e9deb537390ca22-dot-asia-east1.pipelines.googleusercontent.com/#/start`, ENDPOINT value becomes `1e9deb537390ca22-dot-asia-east1.pipelines.googleusercontent.com`.

> **참고: 아래에서 엔드포인트 값을 설정해야 합니다.**

In [ ]:
# This refers to the KFP cluster endpoint
ENDPOINT='' # Enter your ENDPOINT here.
if not ENDPOINT:
    from absl import logging
    logging.error('Set your ENDPOINT in this cell.')

현재 GCP 프로젝트에서 이미지 이름을 `tfx-pipeline`으로 설정합니다.

In [ ]:
# Docker image name for the pipeline image.
CUSTOM_TFX_IMAGE='gcr.io/' + GOOGLE_CLOUD_PROJECT + '/tfx-pipeline'

이제 파이프라인을 생성할 수 있습니다.

## 2단계. 미리 정의된 템플릿을 프로젝트 디렉터리에 복사

이 단계에서는 미리 정의된 템플릿에서 추가 파일을 복사하여 작업 파이프라인 프로젝트 디렉터리 및 파일을 생성하겠습니다.

You may give your pipeline a different name by changing the `PIPELINE_NAME` below. This will also become the name of the project directory where your files will be put.

In [ ]:
PIPELINE_NAME="my_pipeline"
import os
PROJECT_DIR=os.path.join(os.path.expanduser("~"),"imported",PIPELINE_NAME)

TFX includes the `taxi` template with the TFX python package. If you are planning to solve a point-wise prediction problem, including classification and regresssion, this template could be used as a starting point.

`tfx template copy` CLI 명령은 미리 정의된 템플릿 파일을 프로젝트 디렉토리에 복사합니다.

In [ ]:
!tfx template copy \
  --pipeline-name={PIPELINE_NAME} \
  --destination-path={PROJECT_DIR} \
  --model=taxi

이 노트북의 작업 디렉토리 컨텍스트를 프로젝트 디렉토리로 변경합니다.

In [ ]:
%cd {PROJECT_DIR}

> 참고: 생성된 프로젝트 디렉토리를 클릭하여 왼쪽의 `File Browser`에서 디렉토리를 변경해야 합니다.

## 3단계. 복사한 소스 파일 검색

The TFX template provides basic scaffold files to build a pipeline, including Python source code, sample data, and Jupyter Notebooks to analyse the output of the pipeline. The `taxi` template uses the same *Chicago Taxi* dataset and ML model as the [Airflow Tutorial](https://www.tensorflow.org/tfx/tutorials/tfx/airflow_workshop).

다음은 각 Python 파일에 대한 간략한 소개입니다.

- `pipeline` -이 디렉토리는 파이프라인 정의를 포함합니다.
    - `configs.py` — 파이프라인 러너의 공통 상수를 정의합니다.
    - `pipeline.py` — TFX 컴포넌트 및 파이프라인을 정의합니다.
- `models` - 이 디렉토리는 ML 모델 정의를 포함합니다.
    - `features.py`, `features_test.py` — defines features for the model
    - `preprocessing.py`, `preprocessing_test.py` — `tf::Transform`을 사용하여 전처리 작업을 정의합니다.
    - `estimator` - 이 디렉토리는 Estimator 기반 모델을 포함합니다.
        - `constants.py` — 모델 상수를 정의합니다.
        - `model.py`, `model_test.py` — TF estimator를 사용하여 DNN 모델을 정의합니다.
    - `keras` - 이 디렉토리는 Keras 기반 모델을 포함합니다.
        - `constants.py` — 모델의 상수를 정의합니다.
        - `model.py`, `model_test.py` — Keras를 사용하여 DNN 모델을 정의합니다.
- `local_runner.py`, `kubeflow_runner.py` — 각 오케스트레이션 엔진의 러너를 정의합니다.


You might notice that there are some files with `_test.py` in their name. These are unit tests of the pipeline and it is recommended to add more unit tests as you implement your own pipelines.
You can run unit tests by supplying the module name of test files with `-m` flag. You can usually get a module name by deleting `.py` extension and replacing `/` with `.`.  For example:

In [ ]:
!{sys.executable} -m models.features_test
!{sys.executable} -m models.keras.model_test


## 4단계. 첫 번째 TFX 파이프라인 실행

TFX 파이프라인의 구성 요소는 각 실행에 대한 출력을 [ML 메타 데이터 아티팩트](https://www.tensorflow.org/tfx/guide/mlmd)로 생성하여 저장해야 합니다. KFP 클러스터가 액세스할 수 있는 저장소는 모두 사용 가능합니다. 다음 예제에서는 Google Cloud Storage(GCS)를 사용해 보겠습니다. 기본 GCS 버킷은 자동으로 생성되며, 이름은 `<your-project-id>-kubeflowpipelines-default`입니다.


나중에 파이프라인에서 사용할 수 있도록 샘플 데이터를 GCS 버킷에 업로드하겠습니다.

In [ ]:
!gsutil cp data/data.csv gs://{GOOGLE_CLOUD_PROJECT}-kubeflowpipelines-default/tfx-template/data/taxi/data.csv

`tfx pipeline create` 명령을 사용하여 TFX 파이프라인을 생성해 보겠습니다.

> 참고: KFP용 파이프라인을 만들 때 파이프라인을 실행할 컨테이너 이미지가 필요하며, 실행 시 `skaffold`가 이미지를 구축할 것입니다. 스캐폴드는 도커 허브에서 기본 이미지를 가져오기 때문에 처음 이미지를 빌드할 경우에는 5~10분가량 걸리지만 그 이후에는 시간이 단축됩니다.

In [ ]:
!tfx pipeline create  --pipeline-path=kubeflow_runner.py --endpoint={ENDPOINT} \
--build-image

While creating a pipeline, `Dockerfile` will be generated to build a Docker image. Don't forget to add it to the source control system (for example, git) along with other source files.

참고: `airflow`가 설치되지 않고 `--engine`이 지정되지 않는 경우 `kubeflow`가 자동으로 오케스트레이션 엔진으로 선택됩니다.

이제 `tfx run create` 명령을 사용하여 새로 생성된 파이프라인으로 실행을 시작합니다.

In [ ]:
!tfx run create --pipeline-name={PIPELINE_NAME} --endpoint={ENDPOINT}

또는, KFP 대시보드에서 파이프라인을 실행할 수도 있습니다. 새로 실행할 경우 KFP 대시보드의 실험 아래에 나열됩니다. 실험을 클릭하면 진행 상황을 모니터링하고 실행 중에 생성된 아티팩트를 시각화할 수 있습니다.

However, we recommend visiting the KFP Dashboard. You can access the KFP Dashboard from the Cloud AI Platform Pipelines menu in Google Cloud Console. Once you visit the dashboard, you will be able to find the pipeline, and access a wealth of information about the pipeline.
For example, you can find your runs under the *Experiments* menu, and when you open your execution run under Experiments you can find all your artifacts from the pipeline under *Artifacts* menu.

>Note: If your pipeline run fails, you can see detailed logs for each TFX component in the Experiments tab in the KFP Dashboard.
    
One of the major sources of failure is permission related problems. Please make sure your KFP cluster has permissions to access Google Cloud APIs. This can be configured [when you create a KFP cluster in GCP](https://cloud.google.com/ai-platform/pipelines/docs/setting-up), or see [Troubleshooting document in GCP](https://cloud.google.com/ai-platform/pipelines/docs/troubleshooting).

## 5단계. 데이터 검증을 위한 구성 요소 추가

In this step, you will add components for data validation including `StatisticsGen`, `SchemaGen`, and `ExampleValidator`. If you are interested in data validation, please see [Get started with Tensorflow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started).

> **Double-click to change directory to `pipeline` and double-click again to open `pipeline.py`**. Find and uncomment the 3 lines which add `StatisticsGen`, `SchemaGen`, and `ExampleValidator` to the pipeline. (Tip: search for comments containing `TODO(step 5):`).  Make sure to save `pipeline.py` after you edit it.

You now need to update the existing pipeline with modified pipeline definition. Use the `tfx pipeline update` command to update your pipeline, followed by the `tfx run create` command to create a new execution run of your updated pipeline.


In [ ]:
# Update the pipeline
!tfx pipeline update \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT}
# You can run the pipeline the same way.
!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}

### Check pipeline outputs

Visit the KFP dashboard to find pipeline outputs in the page for your pipeline run. Click the *Experiments* tab on the left, and *All runs* in the Experiments page. You should be able to find the latest run under the name of your pipeline.

## 6단계. 훈련용 구성 요소 추가

In this step, you will add components for training and model validation including `Transform`, `Trainer`, `Resolver`, `Evaluator`, and `Pusher`.

> **Double-click to open `pipeline.py`**. Find and uncomment the 5 lines which add `Transform`, `Trainer`, `Resolver`, `Evaluator` and `Pusher` to the pipeline. (Tip: search for `TODO(step 6):`)

이전처럼 이제 수정된 파이프라인 정의로 기존 파이프라인을 업데이트해야 합니다. 지침은 5단계와 동일합니다. `tfx pipeline update`를 사용하여 파이프라인을 업데이트하고 `tfx run create`를 사용하여 실행을 생성합니다.


In [ ]:
!tfx pipeline update \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT}
!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}

이 실행이 성공적으로 완료되면 AI Platform Pipelines에서 TFX 파이프라인을 처음으로 생성하고 실행하게 됩니다.

**참고:** 모델 코드에서 변경한 사항이 있으면 컨테이너 이미지도 다시 빌드해야 합니다. `pipeline update` 명령의 `--build-image` 플래그를 사용하여 다시 빌드를 트리거할 수 있습니다.

**NOTE:** You might have noticed that every time we create a pipeline run, every component runs again and again even though the input and the parameters were not changed. It is waste of time and resources, and you can skip those executions with pipeline caching. You can enable caching by specifying `enable_cache=True` for the `Pipeline` object in `pipeline.py`.


## 7단계. (*선택 사항*) BigQueryExampleGen 사용하기

[BigQuery](https://cloud.google.com/bigquery) is a serverless, highly scalable, and cost-effective cloud data warehouse. BigQuery can be used as a source for training examples in TFX. In this step, we will add `BigQueryExampleGen` to the pipeline.

> **Double-click to open `pipeline.py`**. Comment out `CsvExampleGen` and uncomment the line which creates an instance of `BigQueryExampleGen`. You also need to uncomment the `query` argument of the `create_pipeline` function.

We need to specify which GCP project to use for BigQuery, and this is done by setting `--project` in `beam_pipeline_args` when creating a pipeline.

> **Double-click to open `configs.py`**. Uncomment the definition of `GOOGLE_CLOUD_REGION`, `BIG_QUERY_WITH_DIRECT_RUNNER_BEAM_PIPELINE_ARGS` and `BIG_QUERY_QUERY`. You should replace the region value in this file with the correct values for your GCP project.

> **참고: 더 진행하기 앞서 `configs.py` 파일에서 GCP 지역을 설정해야 합니다.**

> **Change directory one level up.** Click the name of the directory above the file list. The name of the directory is the name of the pipeline which is `my_pipeline` if you didn't change.

> **Double-click to open `kubeflow_runner.py`**. Uncomment two arguments, `query` and `beam_pipeline_args`, for the `create_pipeline` function.

이제 파이프라인에서 BigQuery를 예제 소스로 사용할 수 있습니다. 이전과 같이 파이프라인을 업데이트하고 5단계 및 6단계와 같이 새 실행을 생성합니다.

In [ ]:
!tfx pipeline update \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT}
!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}

## 8단계. (*선택 사항*) KFP로 데이터 흐름 사용하기

여러 [TFX 구성 요소는 Apache Beam](https://www.tensorflow.org/tfx/guide/beam)을 사용하여 데이터 병렬 파이프라인을 구현하며, 이를 통해 [Google Cloud Dataflow](https://cloud.google.com/dataflow/)를 사용하여 데이터 처리 워크로드를 분산할 수 있습니다. 이 단계에서는 Kubeflow 오케스트레이터를 설정하여 데이터 흐름을 Apache Beam의 데이터 처리 백엔드로 사용하겠습니다.

> **Double-click `pipeline` to change directory, and double-click to open `configs.py`**. Uncomment the definition of `GOOGLE_CLOUD_REGION`, and `DATAFLOW_BEAM_PIPELINE_ARGS`.

> **Change directory one level up.** Click the name of the directory above the file list. The name of the directory is the name of the pipeline which is `my_pipeline` if you didn't change.

> **Double-click to open `kubeflow_runner.py`**. Uncomment `beam_pipeline_args`. (Also make sure to comment out current `beam_pipeline_args` that you added in Step 7.)

이제 파이프라인에서 Dataflow를 사용할 수 있습니다. 파이프라인을 업데이트하고 5단계 및 6단계처럼 실행을 생성합니다.

In [ ]:
!tfx pipeline update \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT}
!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}

[Cloud Console의 Dataflow](http://console.cloud.google.com/dataflow)에서 Dataflow 작업을 찾을 수 있습니다.


## 9단계. (*선택 사항*) KFP로 Cloud AI Platform 훈련 및 예측하기

TFX interoperates with several managed GCP services, such as [Cloud AI Platform for Training and Prediction](https://cloud.google.com/ai-platform/). You can set your `Trainer` component to use Cloud AI Platform Training, a managed service for training ML models. Moreover, when your model is built and ready to be served, you can *push* your model to Cloud AI Platform Prediction for serving. In this step, we will set our `Trainer` and `Pusher` component to use Cloud AI Platform services.

> 파일을 수정하기 전에 먼저 *AI Platform Training &amp; Prediction API*를 사용 설정해야 할 수 있습니다.

> **Double-click `pipeline` to change directory, and double-click to open `configs.py`**. Uncomment the definition of `GOOGLE_CLOUD_REGION`, `GCP_AI_PLATFORM_TRAINING_ARGS` and `GCP_AI_PLATFORM_SERVING_ARGS`. We will use our custom built container image to train a model in Cloud AI Platform Training, so we should set `masterConfig.imageUri` in `GCP_AI_PLATFORM_TRAINING_ARGS` to the same value as `CUSTOM_TFX_IMAGE` above.

> **Change directory one level up, and double-click to open `kubeflow_runner.py`**. Uncomment `ai_platform_training_args` and `ai_platform_serving_args`.

파이프라인을 업데이트하고 5단계 및 6단계처럼 실행을 생성합니다.

In [ ]:
!tfx pipeline update \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT}
!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}

You can find your training jobs in [Cloud AI Platform Jobs](https://console.cloud.google.com/ai-platform/jobs). If your pipeline completed successfully, you can find your model in [Cloud AI Platform Models](https://console.cloud.google.com/ai-platform/models).

## 10단계. 파이프라인에 데이터 수집

We made a pipeline for a model using the Chicago Taxi dataset. Now it's time to put your data into the pipeline.

데이터는 GCS 또는 BigQuery를 포함하여 파이프라인이 액세스할 수 있는 모든 곳에 저장할 수 있습니다. 파이프라인 정의를 수정해야 데이터에 액세스할 수 있습니다.

1. If your data is stored in files, modify the `DATA_PATH` in `kubeflow_runner.py` or `local_runner.py` and set it to the location of your files. If your data is stored in BigQuery, modify `BIG_QUERY_QUERY` in `pipeline/configs.py` to correctly query for your data.
2. Add features in `models/features.py`.
3. Modify `models/preprocessing.py` to [transform input data for training](https://www.tensorflow.org/tfx/guide/transform).
4. Modify `models/keras/model.py` and `models/keras/constants.py` to [describe your ML model](https://www.tensorflow.org/tfx/guide/trainer).
    - You can use an estimator based model, too. Change `RUN_FN` constant to `models.estimator.model.run_fn` in `pipeline/configs.py`.

자세한 소개는 [Trainer 구성 요소 가이드](https://www.tensorflow.org/tfx/guide/trainer)를 참조하세요.

## 정리

이 프로젝트에서 사용한 모든 Google Cloud 리소스를 정리하려면 본 튜토리얼에서 사용한 [Google Cloud 프로젝트를 삭제](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)하면 됩니다.

또는 아래의 각 콘솔로 이동하여 개별 리소스를 정리할 수 있습니다.

- [Google Cloud Storage](https://console.cloud.google.com/storage)
- [Google Container Registry](https://console.cloud.google.com/gcr)
- [Google Kubernetes Engine](https://console.cloud.google.com/kubernetes)
